In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan  3 09:50:34 2022

@author: cumhurKılıç
"""

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss, make_scorer
from scipy.spatial import distance_matrix
import lightgbm as lgbm
import time
np.set_printoptions(threshold=100)
import optuna
from optuna.integration import LightGBMPruningCallback

In [2]:
# X Train
train = pd.read_csv("train_proje.csv",  sep=",")
x_test = pd.read_csv("test_proje.csv",  sep=",")
test_sample= pd.read_csv("test_ids_in_prediction.csv", sep='|',  encoding='latin-1')

list_array = test_sample.to_numpy()

x_test_sample = x_test[x_test["unique_id"].isin(list_array)]

In [3]:
print(train)

                       time_stamp   contentid user_action  sellingprice  \
0        2020-12-02T22:26:14.023Z  39918893.0    favorite       3099.00   
1        2020-12-08T23:15:04.603Z   3558544.0    favorite       3079.00   
2        2020-12-05T16:19:01.157Z  31292729.0    favorite       3999.00   
3            2020-12-05T16:28:00Z   6363103.0       visit       2544.00   
4            2020-12-02T22:26:59Z  39918893.0       visit       3099.00   
...                           ...         ...         ...           ...   
5493263      2020-12-09T11:41:59Z  31440796.0      search         89.50   
5493264      2020-12-09T00:10:16Z  54677237.0      search         99.99   
5493265      2020-12-09T11:40:57Z   2699559.0      search        169.00   
5493266  2020-12-09T00:08:49.379Z  34981546.0    favorite        248.48   
5493267      2020-12-09T10:44:35Z  34981546.0       visit        248.48   

                                              product_name  brand_id  \
0        PerfectCare 600 EW

In [4]:
train.drop_duplicates(keep=False,inplace=True)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
cv = CountVectorizer(max_features = 15)
cv2 = CountVectorizer(max_features = 15)
train['product_name'] = train['product_name'].fillna(" ")
train['brand_name'] = train['brand_name'].fillna(" ")
X_product = cv.fit_transform(train['product_name']).toarray()
X_brand = cv2.fit_transform(train['brand_name']).toarray()
X_product = pd.DataFrame(X_product)
X_brand = pd.DataFrame(X_brand)
X_brand.columns = [f'brandcol{i}' for i in range(0,len(X_brand.T))] 

In [7]:
#Check all type train
class3 = (train.iloc[:, -1] != 'train')*1
print(sum(class3))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['gender'] = le.fit_transform(train['gender'])

print(train['gender'])

train['time_stamp'] = pd.to_datetime(train['time_stamp'], 
 format = '%Y-%m-%dT%H:%M:%SZ', 
 errors = 'coerce')
train['time_hour'] = train['time_stamp'].dt.hour
train['time_day'] = train['time_stamp'].dt.day
train['time_month'] = train['time_stamp'].dt.month


x_test_sample['time_stamp'] = pd.to_datetime(x_test_sample['time_stamp'], 
 format = '%Y-%m-%dT%H:%M:%SZ', 
 errors = 'coerce')
x_test_sample['time_hour'] = x_test_sample['time_stamp'].dt.hour
x_test_sample['time_day'] = x_test_sample['time_stamp'].dt.day
x_test_sample['time_month'] = x_test_sample['time_stamp'].dt.month


means = train.groupby('user_action')['gender'].mean()
train['user_action'] = train['user_action'].map(means)
x_test_sample['user_action'] = x_test_sample['user_action'].map(means)

means = train.groupby('businessunit')['gender'].mean()
train['businessunit'] = train['businessunit'].map(means)
x_test_sample['businessunit'] = x_test_sample['businessunit'].map(means)

means = train.groupby('product_gender')['gender'].mean()
train['product_gender'] = train['product_gender'].map(means)
x_test_sample['product_gender'] = x_test_sample['product_gender'].map(means)



train_drop=train.drop(columns=['time_stamp', 'contentid','product_name','brand_name','Level1_Category_Name','Level2_Category_Name','Level3_Category_Name','type','unique_id','gender'], axis=1)
test_drop=x_test_sample.drop(columns=['time_stamp', 'contentid','product_name','brand_name','Level1_Category_Name','Level2_Category_Name','Level3_Category_Name','type','unique_id','gender'], axis=1)




0
2261275    1
2261276    1
2261277    1
2261278    1
2261279    1
          ..
5493263    1
5493264    1
5493265    1
5493266    1
5493267    1
Name: gender, Length: 869635, dtype: int32


In [8]:
train_drop=train_drop.reset_index()
test_drop=test_drop.reset_index()

In [9]:
X_concat =pd.concat([train_drop, X_product,X_brand],  axis=1)
X_concat =X_concat.iloc[:,1:]

In [10]:
print(X_concat)

        user_action  sellingprice  brand_id  businessunit  product_gender  \
0          0.182245         49.99   15508.0      0.499152        0.570840   
1          0.182245         79.99   15508.0      0.499152        0.570840   
2          0.182245         37.99   15508.0      0.499152        0.570840   
3          0.182245         39.99   15508.0      0.499152        0.570840   
4          0.218726         34.99   15508.0      0.499152        0.570840   
...             ...           ...       ...           ...             ...   
869630     0.226630         89.50   30457.0      0.222149        0.214885   
869631     0.226630         99.99  978383.0      0.688746        0.570840   
869632     0.226630        169.00    8171.0      0.222149        0.214885   
869633     0.104013        248.48   93397.0      0.222879        0.570840   
869634     0.182245        248.48   93397.0      0.222879        0.570840   

        category_id  Level1_Category_Id  Level2_Category_Id  \
0           

In [11]:
X =X_concat
y=train['gender']

In [12]:

def objective_2(trial):
    train_data_2 = lgbm.Dataset(X, label=y) 
    params_2={
        "seed": 2612,
        "boosting_type":"gbdt",
        "metric":"auc",
        "num_iterations": trial.suggest_categorical("num_iterations", [100, 300, 500,750, 1000,1500]),
        "max_depth" :  trial.suggest_int("max_depth", 3, 15),
        "num_leaves" : trial.suggest_int('num_leaves', 20, 2**14),
        "subsample" : trial.suggest_float('subsample', 0.2, 1.0) ,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_threads" : trial.suggest_int("num_threads", 5, 135),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=10),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=10),
        "objective": "binary",
         "is_unbalance" : True,
        "early_stopping_rounds" : 100,
        'verbose': -1
    }
    gbm_2 = lgbm.cv(params_2,
           train_set=train_data_2, nfold=10,early_stopping_rounds=30,
                    verbose_eval=False)
    auc = np.mean((gbm_2["auc-mean"]))
    return auc 



In [13]:

import sklearn.datasets
import sklearn.metrics
sampler= optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler,pruner=optuna.pruners.MedianPruner(
        n_startup_trials=2, n_warmup_steps=5, interval_steps=3
    ),direction="maximize")
study.optimize(objective_2, n_trials=40)
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2022-01-24 16:41:14,506] A new study created in memory with name: no-name-5c4c5a90-1142-4adf-8489-c2e443f64cfc
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated an

C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-01-24 17:43:16,570] Trial 6 finished with value: 0.8532115935838664 and parameters: {'num_iterations': 1500, 'max_depth': 4, 'num_leaves': 3475, 'subsample': 0.6321700343550622, 'learning_rate': 0.07787050805728976, 'num_threads': 87, 'lambda_l1': 20, 'lambda_l2': 10}. Best is trial 2 with value: 0.8990908342273373.
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_roun

KeyboardInterrupt: 

In [14]:
train.describe()

,contentid,user_action,sellingprice,brand_id,businessunit,product_gender,category_id,Level1_Category_Id,Level2_Category_Id,Level3_Category_Id,gender,unique_id,time_hour,time_day,time_month
count,8.696340e+05,869635.000000,857450.000000,8.687680e+05,868768.000000,768722.000000,868768.000000,868768.000000,868768.000000,868768.000000,869635.000000,869635.000000,794236.000000,794236.000000,794236.000000
mean,3.792284e+07,0.196193,244.463721,1.780492e+05,0.196051,0.191173,1199.492739,816.213982,1643.267406,1173.174346,0.196193,2615.116725,14.750175,16.433978,10.902404
std,1.658196e+07,0.034516,1102.906477,3.543551e+05,0.156847,0.180920,912.074292,674.389620,1081.853705,824.282750,0.397116,1672.915480,6.764291,8.793101,0.587195
min,5.197300e+04,0.104013,0.000000,1.000000e+01,0.000000,0.062032,369.000000,368.000000,387.000000,369.000000,0.000000,7.000000,0.000000,1.000000,10.000000
25%,3.249171e+07,0.182245,49.990000,1.423000e+03,0.093718,0.062032,567.000000,522.000000,503.000000,567.000000,0.000000,1257.000000,11.000000,9.000000,11.000000
50%,4.196724e+07,0.182245,89.990000,1.107900e+04,0.141388,0.062032,721.000000,522.000000,1203.000000,975.000000,0.000000,2358.000000,16.000000,17.000000,11.000000
75%,4.955623e+07,0.226630,189.000000,3.639800e+04,0.233625,0.214885,1806.000000,758.000000,2869.000000,1188.000000,0.000000,3746.000000,20.000000,25.000000,11.000000
max,6.780625e+07,0.274096,226320.000000,1.042155e+06,0.688746,0.570840,4577.000000,3186.000000,4481.000000,4591.000000,1.000000,7997.000000,23.000000,31.000000,12.000000


In [15]:
x_test_sample['product_name'] = x_test_sample['product_name'].fillna(" ")
x_test_sample['brand_name'] = x_test_sample['brand_name'].fillna(" ")

X_test_product = cv.transform(x_test_sample['product_name']).toarray()
X_test_brand = cv2.transform(x_test_sample['brand_name']).toarray()
X_test_product = pd.DataFrame(X_test_product)
X_test_brand = pd.DataFrame(X_test_brand)
X_test_brand.columns = [f'brandcol{i}' for i in range(0,len(X_test_brand.T))] 

In [16]:
X_test_concat =pd.concat([test_drop, X_test_product,X_test_brand],  axis=1)
X_test_concat =X_test_concat.iloc[:,1:]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1,stratify=y)

In [18]:
train_data = lgbm.Dataset(X_train,y_train)
valid_data = lgbm.Dataset(X_test,y_test)

config_params = {
    "classification" : {
                "num_iterations": 1000,
                "max_depth" :  15,
                "num_leaves" : 5652,
                "subsample" : 0.712 ,
                "learning_rate":  0.020926937689014154,
                "num_threads" :134,
                "lambda_l1": 0,
                "lambda_l2": 80,
                "seed": 2612,
                "boosting_type":"gbdt",
                "metric":"auc",
                "colsample_bytree" : 0.9,
                "objective": "binary",
                 "is_unbalance" : True,
                "early_stopping_rounds" : 100,

        }
   
    
   }
config_params["classification"].update(study.best_params)
config_params["classification"]["n_estimators"] = study.best_params["num_iterations"]#It is equal to 1000
lgb_model =lgbm.train(config_params["classification"],train_data,valid_sets=[train_data,valid_data],verbose_eval=10)
score = lgb_model.best_score
print(score)


C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Public\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 136493, number of negative: 559215
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045359 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1337
[LightGBM] [Info] Number of data points in the train set: 695708, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.196193 -> initscore=-1.410261
[LightGBM] [Info] Start training from score -1.410261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	training's auc: 0.928762	valid_1's auc: 0.90702
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	training's auc: 0.945367	valid_1's auc: 0.920121
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	training's auc: 0.954474	valid_1's auc: 0.927089
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[310]	training's auc: 0.955694	valid_1's auc: 0.928077
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[410]	training's auc: 0.962754	valid_1's auc: 0.933299
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[510]	training's auc: 0.969383	valid_1's auc: 0.93816
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[610]	training's auc: 0.973645	valid_1's auc: 0.941467
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[710]	training's auc: 0.977073	valid_1's auc: 0.943915
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

In [19]:
import shap

In [ ]:
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, features=X_test, feature_names=X_test.columns)

In [ ]:
y_pred = lgb_model.predict(X_test_concat)
print(y_pred)

In [ ]:
y_pred_f =[]
for x in y_pred:
    y_pred_f.append(1-x)
print(y_pred_f[1])

In [ ]:
temp = x_test_sample['unique_id']

In [ ]:
x_test_sample['unique_id']=temp

In [ ]:
x_test_sample['gender']=y_pred_f

In [ ]:
x_test_sample.head()

In [ ]:
means2 = x_test_sample.groupby('unique_id')['gender'].mean()
q25_data = x_test_sample.groupby('unique_id')['gender'].quantile(0.25)
q75_data = x_test_sample.groupby('unique_id')['gender'].quantile(0.75)
print(means2)
print(q25_data)
print(q75_data)

In [ ]:
deneme =""
sayi=0
for x in range(0,len(means2)):
    sayi=(means2.iloc[x]+q25_data.iloc[x]+q75_data.iloc[x])/3.0
    deneme=deneme+","+"{:.4f}".format(sayi)
        
deneme = deneme[1:]
print(deneme)

In [ ]:
result_lgbm=[
0.7961,0.7743,0.7157,0.7571,0.7166,0.7347,0.3034,0.2270,0.6978,0.7385,0.3075,0.1932,0.7207,0.8462,0.7427,0.7157,0.3735,0.2117,0.7882,0.6839,0.8081,0.5053,0.7020,0.7219,0.7618,0.8208,0.8219,0.7963,0.7908,0.6502,0.8095,0.6508,0.8511,0.6410,0.7625,0.7614,0.4911,0.6377,0.7025,0.6934,0.6910,0.7326,0.6922,0.7148,0.7519,0.7287,0.7733,0.4806,0.7031,0.3302,0.3053,0.8036,0.4914,0.1684,0.4094,0.1904,0.6494,0.6795,0.6828,0.7097,0.6774,0.4144,0.2233,0.6861,0.7515,0.5809,0.7989,0.6372,0.5984,0.6872,0.7681,0.7464,0.7660,0.8407,0.6559,0.6322,0.5829,0.7461,0.8592,0.8190,0.3119,0.6356,0.5275,0.2343,0.7893,0.8359,0.5805,0.8144,0.7684,0.7291,0.6576,0.6946,0.5718,0.7455,0.4746,0.7794,0.7061,0.5332,0.1493,0.6543,0.6625,0.7209,0.7330,0.7204,0.6370,0.4859,0.3261,0.1529,0.6261,0.5293,0.7752,0.7571,0.7639,0.7269,0.5519,0.6175,0.5084,0.7519,0.7826,0.6819,0.6752,0.7444,0.2993,0.6407,0.7009,0.6894,0.8405,0.8120,0.2662,0.3647,0.7804,0.5370,0.3402,0.8152,0.7334,0.6809,0.4728,0.8528,0.6538,0.6852,0.3564,0.5314,0.7924,0.6609,0.3821,0.8016,0.5865,0.6997,0.3603,0.6955,0.5246,0.7393,0.5881,0.8131,0.4376,0.5390,0.8110,0.6967,0.7871,0.4689,0.7710,0.6394,0.7759,0.6998,0.6643,0.7968,0.6419,0.8443,0.8260,0.1568,0.6837,0.2871,0.6243,0.8515,0.7136,0.6476,0.7202,0.6811,0.6616,0.6480,0.5345,0.2174,0.6769,0.3498,0.4561,0.7850,0.6866,0.7353,0.8357,0.7593,0.6507,0.5813,0.5425,0.6225,0.8178,0.3150,0.6273,0.4320,0.7146,0.6547,0.6189,0.8367,0.8430,0.7168,0.5883,0.5737,0.3287,0.7177,0.2186,0.2845,0.7634,0.7518,0.3871,0.7331,0.8105,0.2577,0.6924,0.7139,0.5738,0.8193,0.4688,0.7370,0.3244,0.7487,0.6910,0.3830,0.3148,0.6073,0.5462,0.3474,0.2708,0.6475,0.4567,0.8542,0.4033,0.8260,0.7833,0.6241,0.6742,0.7300,0.3427,0.7486,0.7810,0.7892,0.7065,0.4585,0.5411,0.2709,0.7256,0.5943,0.5745,0.1834,0.7928,0.8595,0.1699,0.4514,0.7207,0.7805,0.4136,0.3512,0.6974,0.6170,0.4623,0.6534,0.6635,0.7859,0.4549,0.6484,0.7122,0.3519,0.6387,0.6103,0.7241,0.4739,0.7564,0.7332,0.6883,0.7020,0.6956,0.6436,0.6634,0.8109,0.2175,0.6715,0.4147,0.2557,0.7707,0.7595,0.4566,0.7585,0.3587,0.2139,0.2950,0.7098,0.4113,0.4047,0.6580,0.4159,0.3507,0.7479,0.7193,0.6581,0.7583,0.7053,0.8194,0.6109,0.7026,0.8447,0.8656,0.6487,0.8060,0.5776,0.1824,0.6723,0.6834,0.2222,0.7919,0.2963,0.6113,0.2225,0.7468,0.5143,0.4375,0.4731,0.7584,0.7074,0.2836,0.5766,0.6574,0.5181,0.4525,0.7636,0.4183,0.6800,0.5454,0.7252,0.6878,0.6595,0.2333,0.6825,0.6886,0.7354,0.6900,0.7786,0.7224,0.5358,0.7940,0.3556,0.6156,0.8060,0.3205,0.2784,0.8728,0.7307,0.4445,0.8210,0.6222,0.6886,0.7417,0.7654,0.8652,0.8349,0.2323,0.8017,0.6734,0.7837,0.3110,0.7040,0.6858,0.4667,0.7314,0.7317,0.8109,0.8018,0.6321,0.6838,0.7515,0.8691,0.3309,0.6955,0.7403,0.7514,0.2854,0.8024,0.7755,0.6767,0.3244,0.8522,0.8894,0.7096,0.3521,0.8150,0.4825,0.8127,0.6895,0.7950,0.7917,0.7549,0.7897,0.7320,0.4703,0.7892,0.6894,0.7161,0.7968,0.8219,0.1109,0.2435,0.5894,0.2847,0.2960,0.3328,0.4729,0.4604,0.7798,0.2747,0.6605,0.2210,0.5678,0.5734,0.8178,0.7569,0.7486,0.8059,0.2255,0.2677,0.2697,0.1882,0.7148,0.6873,0.6250,0.2779,0.3304,0.8138,0.5678,0.7333,0.7254,0.5658,0.4018,0.7561,0.2143,0.6897,0.7366,0.4194,0.6968,0.7630,0.3164,0.8074,0.2451,0.3692,0.6816,0.5980,0.2934,0.7675,0.7064,0.7071,0.8003,0.8147,0.7245,0.6687,0.8016,0.4469,0.6786,0.6930,0.7658,0.8565,0.3881,0.5068,0.3550,0.7231,0.7159,0.5002,0.2727,0.6826,0.6034,0.6493,0.5497,0.6931,0.2394,0.7874,0.3648,0.7148,0.8826,0.2436,0.3210,0.2814,0.6727,0.7662,0.6258,0.3190,0.1660,0.6070,0.1613,0.6682,0.4731,0.8330,0.7565,0.4368,0.4502,0.2263,0.7897,0.7181,0.7575,0.5479,0.1912,0.7572,0.5939,0.8106,0.7124,0.3704,0.7627,0.8113,0.6434,0.3022,0.4256,0.4467,0.2097,0.9384,0.5983,0.1834,0.4941,0.4602,0.3119,0.1487,0.7408,0.7958,0.6277,0.8295,0.7115,0.6913,0.1687,0.7236,0.6354,0.4923,0.8161,0.6175,0.5044,0.1818,0.5929,0.6543,0.3241,0.8427,0.1772,0.3686,0.6530,0.3345,0.1949,0.5727,0.3666,0.4075,0.7068,0.6869,0.5753,0.8148,0.6905,0.7521,0.1518,0.5920,0.4551,0.7234,0.8622,0.2266,0.4769,0.7554,0.2107,0.7807,0.7025,0.5015,0.3222,0.6737,0.7473,0.4911,0.2301,0.2026,0.7642,0.6149,0.4741,0.6364,0.8153,0.7166,0.8129,0.1426,0.7240,0.7864,0.4794,0.7894,0.8323,0.6221,0.7459,0.6455,0.5960,0.7774,0.6239,0.2216,0.6033,0.3838,0.7810,0.5182,0.2629,0.7312,0.6728,0.8671,0.4881,0.7419,0.3807,0.4424,0.6135,0.7451,0.8437,0.6801,0.6948,0.7652,0.2016,0.6096,0.1784,0.7693,0.6505,0.3345,0.1950,0.1314,0.6921,0.8759,0.6726,0.7008,0.7574,0.4531,0.2278,0.8013,0.6257,0.7775,0.6890,0.7449,0.5240,0.2232,0.7107,0.3125,0.2131,0.2647,0.6611,0.6956,0.7832,0.7410,0.7207,0.3875,0.7275,0.2437,0.4246,0.6547,0.7574,0.5813,0.7637,0.2740,0.2404,0.5565,0.7083,0.7579,0.7913,0.2791,0.6673,0.8012,0.5853,0.6636,0.3611,0.5587,0.2527,0.3708,0.7624,0.8055,0.8391,0.8543,0.1981,0.6950,0.6371,0.2789,0.5472,0.6074,0.8612,0.5790,0.3465,0.4956,0.5481,0.3669,0.4999,0.7941,0.7239,0.5707,0.4021,0.5719,0.8640,0.7021,0.3852,0.3702,0.5172,0.5949,0.7819,0.7012,0.7288,0.8239,0.3160,0.6731,0.7355,0.6805,0.4251,0.8449,0.6531,0.6791,0.7015,0.2717,0.8049,0.4935,0.6924,0.4711,0.6824,0.3060,0.6742,0.7193,0.7472,0.6803,0.5165,0.5550,0.6691,0.8058,0.8034,0.2392,0.6672,0.7488,0.3833,0.7614,0.3444,0.2753,0.7307,0.4969,0.5702,0.6452,0.5966,0.5963,0.8235,0.7414,0.7102,0.7487,0.7024,0.7271,0.6948,0.6773,0.7317,0.1755,0.3330,0.8165,0.5479,0.7428,0.6764,0.3902,0.5754,0.7084,0.2756,0.7914,0.8367,0.2614,0.5912,0.7943,0.8432,0.7470,0.2443,0.6942,0.8712,0.7911,0.6331,0.3511,0.6790,0.6154,0.3146,0.7429,0.8171,0.2214,0.6571,0.1416,0.3220,0.4146,0.6211,0.5422,0.8178,0.3759,0.6907,0.5845,0.8240,0.4097,0.3641,0.5241,0.8324,0.2045,0.1061,0.7820,0.3726,0.7441,0.5162,0.6350,0.7132,0.3961,0.8245,0.6896,0.3932,0.2866,0.6319,0.1414,0.5465,0.8017,0.5432,0.5854,0.7618,0.7957,0.8406,0.2473,0.6590,0.6389,0.4939,0.6521,0.8229,0.8321,0.6276,0.8141,0.8058,0.7146,0.7721,0.7084,0.5900,0.4541,0.6755,0.6448,0.4664,0.8180,0.6786,0.7260,0.6310,0.6254,0.5000,0.6820,0.4649,0.2686,0.6891,0.0901,0.7846,0.5033,0.7216,0.7291,0.7454,0.8516,0.8611,0.6249,0.7825,0.6291,0.4640,0.6228,0.5607,0.2687,0.3290,0.8737,0.6885,0.2180,0.7321,0.7318,0.7287,0.7939,0.4313,0.7173,0.3737,0.2739,0.2385,0.8469,0.6515,0.5116,0.8221,0.5346,0.7716,0.5640,0.5551,0.7542,0.7385,0.7002,0.2568,0.3655,0.3994,0.1860,0.7182,0.6227,0.5410,0.3732,0.8327,0.2931,0.3718,0.4821,0.7900,0.4553,0.1969,0.5412,0.7538,0.6624,0.6940,0.2116,0.4330,0.7865,0.3527,0.7376,0.6133,0.1941,0.2605,0.5560,0.3198,0.3362,0.4021,0.7008,0.3741,0.2002,0.8477,0.8144,0.8790,0.6377,0.7860,0.6289,0.4195,0.7639,0.6898,0.2371,0.7402,0.7602,0.4530,0.2095,0.5959,0.3157,0.4108,0.4176,0.7270,0.8404,0.8796,0.7601,0.4212,0.6590,0.7519,0.1534,0.6980,0.6675,0.7482,0.2008,0.4618,0.8928,0.7187,0.6745,0.6796,0.5808,0.6607,0.4548,0.6892,0.5344,0.2633,0.8156,0.7825,0.5522,0.7473,0.8145,0.8541,0.5721,0.8481,0.3062,0.4091,0.6529,0.6931,0.8459,0.8004,0.2014,0.8652,0.3405,0.5046,0.2216,0.3037,0.8755,0.7961,0.5444,0.6423,0.6726,0.6752,0.2391,0.7949,0.6164,0.6521,0.2464,0.8822,0.2306,0.6457,0.3953,0.6497,0.7876,0.6434,0.8670,0.6668,0.6481,0.3383,0.7217,0.7471,0.6093,0.7668,0.7292,0.7820,0.6737,0.4745,0.8099,0.7663,0.7355,0.4422,0.2977,0.6844,0.4669,0.2179,0.8032,0.5685,0.8720,0.6144,0.1285,0.4671,0.6416,0.2889,0.7678,0.2280,0.7471,0.3561,0.7376,0.1871,0.5158,0.1160,0.2033,0.2537,0.8096,0.6879,0.7792,0.7954,0.1949,0.6885,0.5040,0.5511,0.1921,0.7607,0.5448,0.4312,0.3424,0.3000,0.4522,0.1954,0.2296,0.7581,0.7766,0.9512,0.8918,0.7868,0.2885,0.6691,0.7061,0.8292,0.6559,0.6876,0.1943,0.5269,0.6406,0.5592,0.1959,0.7824,0.8225,0.5203,0.8368,0.8254,0.6492,0.8359,0.6205,0.7695,0.3801,0.6751,0.6251,0.5302,0.7124,0.6667,0.8607,0.4373,0.6510,0.2661,0.0905,0.4608,0.2926,0.5729,0.5761,0.1775,0.5903,0.4695,0.2201,0.3115,0.3931,0.4863,0.2952,0.2695,0.4310,0.7220,0.6349,0.4196,0.8081,0.5605,0.1501,0.1579,0.3456,0.5936,0.1611,0.3997,0.4428,0.1882,0.1804,0.1921,0.7198,0.7368,0.5605,0.4288,0.4662,0.1755,0.6825,0.5394,0.4686,0.1538,0.8243,0.5274,0.2796,0.4040,0.6204,0.1850,0.5812,0.7342,0.5125,0.2222,0.8108,0.2531,0.6970,0.3392,0.6722,0.8222,0.6446,0.7846,0.2396,0.4810,0.5974,0.5773,0.3246,0.7348,0.5476,0.8013,0.1764,0.6695,0.5642,0.6053,0.1702,0.6759,0.5864,0.6858,0.5363,0.7790,0.8426,0.2687,0.3207,0.3739,0.6214,0.7902,0.7627,0.5513,0.9064,0.4795,0.9227,0.7073,0.6183,0.2383,0.8307,0.2124,0.7594,0.4669,0.8268,0.2617,0.5659,0.2219,0.8330,0.6817,0.4664,0.4025,0.3337,0.6622,0.6051,0.8230,0.2072,0.8170,0.3625,0.2369,0.7134,0.1746,0.2032,0.5316,0.2237,0.6951,0.5660,0.5885,0.2506,0.6081,0.6342,0.4919,0.4031,0.5334,0.2429,0.4594,0.5457,0.6220,0.7249,0.8338,0.3451,0.8490,0.6103,0.7841,0.5109,0.7559,0.6138,0.5435,0.6816,0.6579,0.7668,0.6218,0.4458,0.1803,0.3402,0.3899,0.2408,0.4495,0.5051,0.1794,0.2259,0.6361,0.5028,0.7225,0.6816,0.2916,0.7312,0.1487,0.1129,0.1764,0.2129,0.6974,0.6788,0.2805,0.5578,0.7105,0.8510,0.6174,0.6116,0.5380,0.6796,0.6596,0.3801,0.6547,0.3669,0.6983,0.5064,0.5147,0.2646,0.6557,0.6430,0.9123,0.3608,0.3131,0.3418,0.1119,0.8764,0.8630,0.4500,0.6808,0.1839,0.4144,0.6035,0.6711,0.5709,0.1910,0.7487,0.3462,0.7781,0.8059,0.2555,0.5541,0.5716,0.5297,0.7383,0.2205,0.2154,0.7903,0.6355,0.6439,0.2852,0.2229,0.2126,0.2000,0.6363,0.4035,0.6050,0.7091,0.6691,0.9163,0.5156,0.2806,0.6772,0.1815,0.7582,0.2122,0.6002,0.1619,0.7337,0.5768,0.7586,0.1860,0.2752,0.7555,0.3069,0.6327,0.7272,0.5165,0.1626,0.5548,0.9421,0.8523,0.3244,0.8405,0.2790,0.2660,0.4213,0.5583,0.7598,0.3999,0.8240,0.5464,0.2869,0.5053,0.7060,0.3840,0.6323,0.2280,0.4339,0.3619,0.8459,0.4143,0.6555,0.1837,0.7197,0.9261,0.7388,0.7440,0.3372,0.8421,0.3178,0.3041,0.5299,0.4942,0.3863,0.4870,0.5622,0.3169,0.5451,0.8779,0.2412,0.1515,0.2648,0.6490,0.4453,0.3287,0.7546,0.1878,0.3849,0.7012,0.6905,0.4687,0.7167,0.6369,0.2247,0.2673,0.4259,0.4141,0.5046,0.3044,0.5328,0.4484,0.4302,0.2754,0.7314,0.2710,0.1755,0.1347,0.1016,0.5308,0.7695,0.5343,0.2095,0.1614,0.3965,0.3802,0.2197,0.2281,0.3645,0.3441,0.3254,0.3139,0.7757,0.3935,0.4704,0.3465,0.2056,0.4534,0.6508,0.3290,0.3306,0.4010,0.2057,0.4972,0.3043,0.4496,0.2272,0.7172,0.4543,0.2086,0.2328,0.3052,0.6901,0.5753,0.3548,0.5559,0.6998,0.2648,0.3478,0.4662,0.2114,0.3225,0.6068,0.7619,0.7651,0.4207,0.3803,0.4085,0.4296,0.6247,0.7039,0.5241,0.1754,0.1982,0.6391,0.4014,0.2242,0.2846,0.4881,0.4907,0.3051,0.1531,0.7005,0.5755,0.2571,0.1808,0.5745,0.3541,0.8199,0.8077,0.1339,0.2591,0.7160,0.1498,0.4718,0.3345,0.8815,0.5893,0.3638,0.2795,0.6023,0.5230,0.6670,0.7028,0.4689,0.4335,0.8119,0.1946,0.3438,0.7914,0.8830,0.7847,0.5730,0.5132,0.6280,0.7113,0.4871,0.6690,0.5084,0.8505,0.3052,0.7088,0.7876,0.3953,0.5277,0.6320,0.8465,0.4527,0.5684,0.5185,0.7047,0.8275,0.7794,0.4163,0.8340,0.6411,0.6595,0.8493,0.6786,0.4699,0.3196,0.2323,0.7044,0.1878,0.8218,0.2205,0.7519,0.5647,0.6500,0.5373,0.5743,0.8331,0.5091,0.6875,0.8696,0.7051,0.5850,0.2833,0.7493,0.2332,0.7379,0.8514,0.1825,0.7648,0.1597,0.1892,0.6677,0.2423,0.5622,0.7940,0.8520,0.6835,0.7185,0.1566,0.1491,0.3006,0.5643,0.1968,0.1986,0.1541,0.7096,0.2391,0.2266,0.8686,0.1989,0.7019,0.5827,0.8176,0.8001,0.2321,0.1891,0.5976,0.2028,0.7893,0.6451,0.7921,0.6277,0.2140,0.4698,0.4384,0.8186,0.6657,0.8009,0.9194,0.1559,0.8567,0.2770,0.2228,0.7188,0.2660,0.7991,0.2111,0.4010,0.8807,0.1969,0.8343,0.4281,0.6932,0.7795,0.7523,0.8142,0.1353,0.8567,0.3177,0.6659,0.5090,0.2565,0.7798,0.7144,0.8374,0.5881,0.9010,0.5563,0.1987,0.2502,0.3653,0.1462,0.6659,0.3966,0.1902,0.7371,0.7506,0.3961,0.6082,0.7831,0.1960,0.3416,0.6815,0.1702,0.6767,0.5553,0.7037,0.4041,0.8108,0.3048,0.7245,0.3449,0.2080,0.5288,0.4769,0.4155,0.1002,0.1663,0.5667,0.8967,0.8128,0.7155,0.8325,0.2309,0.6711,0.7045,0.1087,0.2102,0.3762,0.8100,0.6910,0.5368,0.7426,0.7737,0.8824,0.7343,0.7285,0.6465,0.3025,0.8551,0.6703,0.7871,0.8042,0.1459,0.1046,0.8569,0.5990,0.3433,0.7672,0.6376,0.8518,0.8536,0.6778,0.8246,0.6859,0.7845,0.8871,0.7547,0.5460,0.9209,0.2469,0.2291,0.1746,0.1238,0.7998,0.1356,0.8852,0.8140,0.1685,0.3139,0.9107,0.2713,0.8397,0.7053,0.7875,0.6244,0.7682,0.2848,0.1663,0.6792,0.9395,0.2201,0.7080,0.3288,0.2041,0.5869,0.3824,0.3026,0.8679,0.7512,0.3276,0.6299,0.6388,0.8044,0.6303,0.7154,0.4071,0.7078,0.8897,0.5171,0.7751,0.6946,0.6121,0.6657,0.8007,0.5160,0.8513,0.5990,0.6074,0.3078,0.1866,0.1992,0.1577,0.5513,0.8737,0.7807,0.1362,0.7797,0.7750,0.2623,0.7936,0.7123,0.3769,0.2998,0.1922,0.2381,0.8019,0.8902,0.8167,0.2361,0.7163,0.6076,0.5132,0.4073,0.1962,0.2418,0.3069,0.8610,0.4624,0.1694,0.8224,0.8754,0.2835,0.5315,0.4268,0.7807,0.1484,0.6166,0.5955,0.7677,0.4213,0.2436,0.2905,0.2384,0.3217,0.8366,0.6480,0.1975,0.4683,0.7851,0.8787,0.9014,0.2004,0.6225,0.1749,0.8790,0.8983,0.7409,0.8076,0.7464,0.5333,0.8756,0.6764,0.8209,0.4916,0.8884,0.6134,0.6942,0.7025,0.1339,0.2372,0.2065,0.2773,0.7280,0.2950,0.2225,0.7314,0.2929,0.5686,0.3080,0.3016,0.5645,0.2895,0.1176,0.7353,0.1795,0.6546,0.1795,0.5132,0.3160,0.3719,0.3953,0.5282,0.4015,0.5265,0.2916,0.5663,0.4218,0.4033,0.6908,0.5436,0.4709,0.5174,0.2793,0.6398,0.3087,0.6785,0.3530,0.6968,0.5236,0.6345,0.6358,0.4019,0.2347,0.4885,0.5949,0.6829,0.5206,0.3528,0.5140,0.3379,0.5323,0.3009,0.6214,0.6305,0.4302,0.2412,0.5421,0.3013,0.4477,0.4100,0.3301,0.5773,0.6188,0.4009,0.4126,0.3322,0.3388,0.3610,0.6187,0.6178,0.3591,0.4162,0.3486,0.6025,0.3629,0.1059,0.0759,0.0702,0.5310,0.2130,0.2704,0.7008,0.7789,0.7365,0.7137,0.5449,0.8623,0.7946,0.6738,0.7598,0.6188,0.7673,0.6874,0.7265,0.6674,0.6748,0.4661,0.7789,0.5276,0.5954,0.7004,0.8575,0.6464,0.6308,0.5889,0.5926,0.7647,0.4203,0.7436,0.7639,0.6300,0.2959,0.5263,0.8263,0.8568,0.8176,0.9548,0.6492,0.5992,0.4666,0.3927,0.3768,0.4436,0.8194,0.2538,0.4329,0.2864,0.8423,0.8312,0.1913,0.7006,0.2387,0.7616,0.6846,0.5874,0.1421,0.7385,0.0643,0.7997,0.1416,0.5732,0.8493,0.5295,0.1944,0.3775,0.6647,0.1424,0.7347,0.4780,0.3649,0.7330,0.1832,0.7469,0.2032,0.2695,0.8666,0.6889,0.1092,0.1931,0.6498,0.2323,0.5865,0.6556,0.5765,0.4946,0.8684,0.9050,0.8891,0.5982,0.4468,0.3691,0.6138,0.3835,0.1503,0.0620,0.1813,0.4961,0.5610,0.2246,0.2802,0.2397,0.1649,0.6712,0.4723,0.2166,0.4437,0.3636,0.4859,0.2809,0.2426,0.2452,0.3470,0.6005,0.3290,0.2841,0.5867,0.5071,0.5538,0.1564,0.3423,0.5619,0.4714,0.6925,0.4656,0.5930,0.6221,0.7174,0.5310,0.7406,0.5060,0.4553,0.5723,0.4378,0.4024,0.4391,0.2490,0.3753,0.3259,0.6524,0.8212,0.6683,0.9174,0.6043,0.7276,0.7426,0.8397,0.8172,0.5974,0.6953,0.6240,0.4529,0.6158,0.8391,0.8634,0.2639,0.1169,0.2923,0.2771,0.2313,0.4632,0.6012,0.1499,0.2289,0.2912,0.2874,0.4127,0.2498,0.5461,0.8131,0.8066,0.4388,0.6298,0.4730,0.4067,0.3564,0.3999,0.5115,0.6165,0.6809,0.3957,0.6051,0.6400,0.4284,0.4846,0.4101,0.6133,0.4035,0.4414,0.3468,0.4918,0.6059,0.5056,0.6770,0.8189,0.5557,0.5684,0.6813,0.6545,0.7451,0.4879,0.5343,0.6651,0.4654,0.4804,0.3474,0.7308,0.6496,0.5267,0.7391,0.4893,0.6868,0.3204,0.2940,0.3947,0.0885,0.6357,0.3481,0.4265,0.8286,0.3712,0.3870,0.5802,0.5194,0.4235,0.4700,0.1674,0.3877,0.4444,0.8284,0.7463,0.1541,0.1997,0.8504,0.6131,0.6136,0.1164,0.5064,0.2333,0.2077,0.1487,0.2842,0.4630,0.6456,0.5022,0.6748,0.2146,0.4155,0.1371,0.8389,0.7329,0.9016,0.1510,0.8438,0.1437,0.7302,0.5250,0.9052,0.7839,0.1646,0.7603,0.2717,0.5132,0.1548,0.5189,0.2470,0.2195,0.4054,0.6203,0.2489,0.3140,0.6508,0.4322,0.4589,0.7621,0.4496,0.8555,0.3587,0.3643,0.7747,0.7508,0.4676,0.9043,0.4045,0.6870,0.7492,0.5694,0.2327,0.1540,0.4370,0.5517,0.2772,0.3139,0.5157,0.1815,0.8244,0.5158,0.2126,0.2598,0.1893,0.3681,0.2819,0.4646,0.5615,0.6692,0.5537,0.5836,0.4404,0.7451,0.6320,0.9129,0.8779,0.5978,0.3140,0.1816,0.3315,0.2437,0.4629,0.2641,0.6849,0.4834,0.1430,0.3720,0.2981,0.3634,0.1098,0.4551,0.2944,0.5458,0.2996,0.6494,0.4900,0.7549,0.2379,0.8262,0.4726,0.2070,0.2524,0.6945,0.7586,0.1168,0.1873,0.3203,0.9229,0.0894,0.8417,0.5468,0.6239,0.4620,0.6255,0.7775,0.5353,0.9249,0.6198,0.8874,0.9765,0.1795,0.1572,0.1795,0.3604,0.3422,0.4920,0.2677,0.2201,0.5696,0.2871,0.1449,0.0425,0.9221,0.3877,0.3552,0.3308,0.3180,0.7942,0.1394,0.3399,0.2981,0.5154,0.6790,0.7821,0.8617,0.5059,0.0786,0.6280,0.6153,0.5150,0.4949,0.5800,0.5173,0.8378,0.8226,0.3603,0.1007,0.5172,0.1881,0.4425,0.1581,0.2392,0.6377,0.8697,0.1420,0.1713,0.1808,0.8158,0.5753,0.7093,0.5519,0.4299,0.4769,0.6221,0.7079,0.2990,0.6317,0.7214,0.6569,0.4250,0.4921,0.4047,0.4426,0.3370,0.2582,0.0668,0.3119,0.1966,0.3008,0.1855,0.1149,0.8241,0.1572,0.2682,0.2456,0.5746,0.4464,0.6292,0.9535,0.1398,0.5740,0.4184,0.2928,0.4813,0.6508,0.7810,0.5438,0.3013,0.8048,0.3245,0.1986,0.9505,0.6356,0.1781,0.3835,0.0922,0.8892,0.6679,0.4922,0.4168,0.9130,0.3150,0.4107,0.3547,0.1407,0.1924,0.1356,0.3651,0.2518,0.4595,0.2821,0.1570,0.2478,0.4086,0.2352,0.3788,0.7673,0.7129,0.2359,0.5432,0.3800,0.4048,0.3116,0.2840,0.2375,0.4687,0.4444,0.4032,0.8426,0.1255,0.0731,0.8503,0.3581,0.8770,0.9429,0.8648,0.7334,0.8900,0.6040,0.5973,0.4672,0.9330,0.2279,0.8286,0.0395,0.4884,0.2963,0.6722   

]


In [ ]:
result_mıl=[
0.9603,0.9497,0.9380,0.9761,0.9813,0.9726,0.2123,0.2389,0.9676,0.9732,0.1318,0.0994,0.9684,0.9793,0.9068,0.9400,0.2021,0.1174,0.9560,0.8804,0.9720,0.6563,0.9720,0.9455,0.9783,0.9786,0.9515,0.9801,0.9767,0.8616,0.9777,0.8757,0.9780,0.9025,0.9631,0.9315,0.5139,0.5844,0.9719,0.9262,0.9678,0.9510,0.9729,0.6879,0.9685,0.9150,0.9614,0.1302,0.6321,0.3160,0.1773,0.9575,0.6758,0.0758,0.1071,0.0889,0.9319,0.9161,0.9677,0.9627,0.9188,0.5482,0.1147,0.8637,0.8823,0.7764,0.9795,0.8820,0.8886,0.9764,0.9818,0.9649,0.9203,0.9662,0.8761,0.9206,0.8418,0.9767,0.9641,0.9583,0.1516,0.8797,0.6777,0.0901,0.9682,0.8832,0.6765,0.9797,0.9745,0.8192,0.9775,0.8954,0.8040,0.9818,0.4095,0.9589,0.9552,0.4988,0.1999,0.9619,0.9480,0.9723,0.9599,0.9695,0.6359,0.5377,0.2749,0.3407,0.8931,0.2312,0.9554,0.9748,0.9520,0.9493,0.8074,0.8790,0.7029,0.9729,0.9692,0.8688,0.9221,0.9809,0.3297,0.7099,0.9187,0.9466,0.9692,0.9742,0.0766,0.1117,0.9725,0.3395,0.0907,0.9763,0.8684,0.9322,0.5859,0.9209,0.7110,0.9305,0.1636,0.7789,0.9761,0.9425,0.5277,0.9753,0.8829,0.9448,0.1371,0.8562,0.8757,0.9528,0.9212,0.9777,0.1497,0.8902,0.9580,0.9210,0.9528,0.7287,0.9645,0.9222,0.9688,0.9638,0.9520,0.9773,0.7762,0.9623,0.8753,0.1959,0.9310,0.2222,0.6866,0.9777,0.9701,0.8816,0.9060,0.9578,0.6741,0.8896,0.3010,0.1323,0.9266,0.2208,0.2110,0.9800,0.8932,0.9755,0.9713,0.8817,0.9663,0.5726,0.6980,0.8668,0.8107,0.1808,0.9267,0.2041,0.9593,0.8378,0.7600,0.9710,0.9786,0.9546,0.8741,0.8126,0.1732,0.7101,0.1420,0.0955,0.9709,0.9457,0.2713,0.9638,0.9337,0.6126,0.9521,0.9475,0.4173,0.9610,0.9488,0.7676,0.3415,0.9237,0.9412,0.1003,0.1272,0.9632,0.4931,0.1215,0.0999,0.9203,0.6351,0.9776,0.1032,0.9702,0.9702,0.6070,0.9795,0.9696,0.1018,0.9406,0.9504,0.9797,0.9276,0.6106,0.6749,0.1072,0.9609,0.8445,0.6263,0.0838,0.9648,0.9698,0.0951,0.7079,0.9662,0.9720,0.1339,0.1071,0.9590,0.7471,0.1156,0.9600,0.9488,0.9725,0.2133,0.9496,0.9269,0.1821,0.9264,0.7445,0.8772,0.1114,0.9489,0.8812,0.9535,0.9122,0.9412,0.9121,0.9770,0.9765,0.0598,0.9415,0.3171,0.0904,0.9734,0.9513,0.7747,0.9708,0.1972,0.0697,0.1160,0.9518,0.1932,0.1844,0.8503,0.1285,0.1831,0.9736,0.9688,0.8597,0.9731,0.9508,0.9458,0.7403,0.9537,0.9698,0.9739,0.9488,0.9815,0.9465,0.1936,0.9337,0.9664,0.1273,0.9748,0.3672,0.7686,0.1902,0.6677,0.7162,0.1229,0.5007,0.9322,0.8964,0.1634,0.4655,0.8492,0.2603,0.1893,0.9723,0.2052,0.9677,0.3046,0.8477,0.9379,0.9645,0.0723,0.9576,0.8806,0.9384,0.8610,0.9793,0.9584,0.8116,0.9527,0.1528,0.8354,0.9805,0.2950,0.1683,0.9690,0.9220,0.1308,0.9678,0.6556,0.9647,0.9748,0.9802,0.9724,0.9378,0.3207,0.9345,0.9208,0.9730,0.1033,0.9537,0.9544,0.2600,0.8787,0.9282,0.9796,0.9725,0.9106,0.9377,0.9399,0.9604,0.7467,0.8116,0.8210,0.9424,0.0842,0.4709,0.9652,0.9326,0.1172,0.9693,0.9606,0.8847,0.1750,0.9776,0.2653,0.9759,0.9069,0.9746,0.8628,0.9803,0.9782,0.9487,0.6845,0.9764,0.9672,0.9034,0.9596,0.9394,0.1798,0.1115,0.7297,0.0807,0.0702,0.1717,0.7582,0.5708,0.8427,0.1099,0.9380,0.1181,0.1107,0.7022,0.9809,0.8600,0.9285,0.9586,0.1184,0.1352,0.2107,0.0976,0.9379,0.7945,0.9164,0.3251,0.1438,0.9772,0.5696,0.9579,0.9424,0.6860,0.5067,0.8978,0.1264,0.8670,0.9617,0.1623,0.9589,0.7158,0.0788,0.9726,0.0622,0.1925,0.8737,0.9693,0.2623,0.9579,0.8932,0.9181,0.9736,0.9800,0.9661,0.9708,0.9773,0.3176,0.9486,0.9100,0.9475,0.9326,0.0952,0.8122,0.1345,0.9779,0.6287,0.2640,0.0742,0.9600,0.6121,0.8442,0.7144,0.8448,0.1573,0.9033,0.1192,0.9385,0.9665,0.1167,0.1210,0.2089,0.9451,0.9503,0.7756,0.0876,0.0898,0.8726,0.2072,0.9287,0.7465,0.9719,0.9642,0.2623,0.1602,0.1242,0.9713,0.9472,0.9747,0.3831,0.1513,0.9735,0.4268,0.9722,0.9528,0.6382,0.8808,0.9059,0.9116,0.1408,0.1397,0.1079,0.1214,0.6211,0.8948,0.0916,0.5963,0.1722,0.0898,0.2073,0.9728,0.9582,0.8904,0.9737,0.9752,0.9212,0.0966,0.9706,0.6399,0.1371,0.9784,0.8770,0.6703,0.1521,0.6384,0.7348,0.0976,0.9771,0.3370,0.2754,0.9154,0.1374,0.0748,0.9248,0.3556,0.2302,0.9611,0.9100,0.7016,0.9798,0.7276,0.9480,0.1090,0.7690,0.1945,0.8858,0.9700,0.1367,0.4423,0.9272,0.1241,0.9629,0.9485,0.3581,0.2566,0.9357,0.9675,0.2535,0.0880,0.1530,0.9704,0.9485,0.4277,0.7417,0.9797,0.9549,0.9793,0.0814,0.9747,0.9724,0.3836,0.9781,0.9624,0.6856,0.9687,0.8992,0.6698,0.9588,0.8322,0.1030,0.9619,0.6553,0.9762,0.3068,0.1062,0.5868,0.9587,0.9819,0.5280,0.9414,0.0780,0.7331,0.3539,0.9673,0.9720,0.9636,0.9760,0.9517,0.1261,0.7803,0.1011,0.9807,0.6422,0.1209,0.1838,0.2807,0.9620,0.9692,0.9130,0.9688,0.8627,0.1941,0.0835,0.9666,0.9010,0.9740,0.8605,0.9390,0.4535,0.1264,0.9470,0.1739,0.3001,0.0856,0.9122,0.9683,0.9774,0.9706,0.9008,0.5291,0.9520,0.1194,0.2971,0.9747,0.9735,0.7605,0.9759,0.3346,0.1262,0.8139,0.9704,0.9625,0.8010,0.1966,0.9563,0.9112,0.6015,0.8450,0.1568,0.4104,0.1501,0.1845,0.9641,0.9802,0.9691,0.8657,0.1916,0.9571,0.6736,0.1613,0.5912,0.5325,0.9732,0.8131,0.1249,0.5263,0.5378,0.1005,0.3741,0.9628,0.8380,0.8760,0.0931,0.8620,0.5808,0.5690,0.1154,0.2462,0.8246,0.8344,0.8142,0.8431,0.9707,0.9719,0.2337,0.9348,0.9063,0.9367,0.1296,0.9755,0.8376,0.8785,0.8727,0.2212,0.9569,0.5688,0.9362,0.2318,0.9112,0.4053,0.9440,0.9592,0.9330,0.7833,0.4558,0.6988,0.8780,0.6783,0.9649,0.2104,0.9641,0.7404,0.1110,0.9670,0.1218,0.1445,0.9653,0.3139,0.8048,0.8829,0.8106,0.7964,0.9715,0.9795,0.9181,0.9470,0.9040,0.9215,0.6703,0.9280,0.9665,0.2577,0.6115,0.9147,0.6529,0.9549,0.9678,0.1542,0.5883,0.9520,0.1880,0.9084,0.9753,0.3591,0.5100,0.9458,0.9739,0.9773,0.1016,0.6665,0.9712,0.9383,0.8674,0.3842,0.9153,0.5921,0.1129,0.9574,0.9733,0.1434,0.6539,0.1552,0.1495,0.6531,0.9271,0.7545,0.9428,0.2313,0.9555,0.8292,0.9757,0.2005,0.2766,0.8220,0.9773,0.0986,0.1680,0.9568,0.3622,0.8298,0.3700,0.2421,0.9377,0.1483,0.9797,0.9075,0.1177,0.0954,0.9387,0.1503,0.6160,0.9776,0.8813,0.7969,0.9779,0.9736,0.9797,0.1316,0.9563,0.7385,0.3766,0.8835,0.9687,0.9470,0.9477,0.9307,0.9706,0.9375,0.7609,0.9026,0.9435,0.7484,0.9349,0.7977,0.2819,0.9348,0.9560,0.9741,0.6204,0.7251,0.1749,0.8581,0.2120,0.1421,0.7770,0.2155,0.9795,0.6099,0.9759,0.9738,0.9723,0.9527,0.9274,0.7991,0.9247,0.6952,0.5950,0.7920,0.8190,0.1695,0.2621,0.9771,0.8251,0.1374,0.9387,0.8992,0.9696,0.9754,0.5365,0.9726,0.1221,0.1596,0.4455,0.9650,0.8113,0.1945,0.9673,0.6345,0.9664,0.6160,0.5413,0.9800,0.8965,0.9539,0.0743,0.1573,0.2500,0.2191,0.9228,0.7968,0.7657,0.1015,0.9759,0.4997,0.3016,0.2683,0.9780,0.1148,0.1291,0.3779,0.9579,0.9565,0.9733,0.1657,0.0866,0.6919,0.1618,0.7844,0.8187,0.1571,0.1884,0.9217,0.0919,0.1530,0.3132,0.6708,0.1240,0.1175,0.9518,0.9748,0.5902,0.9289,0.9710,0.8097,0.2603,0.9722,0.8445,0.2157,0.9663,0.7366,0.9108,0.1257,0.9040,0.2847,0.6781,0.1494,0.9610,0.9647,0.9086,0.9768,0.4199,0.9195,0.9742,0.1244,0.9733,0.7492,0.8738,0.2000,0.2621,0.9654,0.9689,0.9585,0.8130,0.9219,0.9308,0.1854,0.9723,0.2345,0.1426,0.9635,0.7723,0.8573,0.9766,0.9703,0.9556,0.5566,0.9736,0.5280,0.2501,0.6711,0.8997,0.9736,0.7900,0.1393,0.9459,0.1913,0.2673,0.2096,0.1388,0.9664,0.9688,0.8438,0.8591,0.9585,0.9635,0.1249,0.9758,0.7126,0.8625,0.1931,0.9453,0.1358,0.9706,0.3553,0.9382,0.9772,0.9631,0.9661,0.9250,0.9544,0.1117,0.9727,0.9490,0.8555,0.9571,0.8575,0.8942,0.6147,0.2246,0.8947,0.9591,0.9165,0.3202,0.1130,0.9368,0.6003,0.1197,0.9728,0.8071,0.9596,0.8899,0.2298,0.2155,0.9367,0.2957,0.9759,0.1092,0.9185,0.2213,0.9709,0.0922,0.6222,0.2398,0.2137,0.1232,0.9560,0.9698,0.9400,0.9414,0.2449,0.9262,0.2267,0.6181,0.1064,0.7702,0.8496,0.1120,0.1122,0.0936,0.2433,0.1295,0.0987,0.9205,0.9756,0.7219,0.9539,0.9550,0.2874,0.9354,0.9426,0.9477,0.9671,0.9731,0.1324,0.4694,0.7562,0.7645,0.2188,0.9302,0.9688,0.9392,0.9652,0.9201,0.8498,0.9748,0.7947,0.9507,0.3150,0.8364,0.8969,0.1677,0.7757,0.7768,0.9680,0.2272,0.9363,0.1150,0.2483,0.3285,0.1431,0.6111,0.7096,0.4161,0.7565,0.4428,0.4437,0.2299,0.1391,0.7894,0.0803,0.3705,0.2638,0.9464,0.9240,0.2344,0.9596,0.6791,0.1067,0.1964,0.3861,0.8177,0.2268,0.3927,0.5629,0.1637,0.1840,0.3178,0.9740,0.9779,0.8243,0.2415,0.6753,0.1041,0.9452,0.4788,0.5862,0.1575,0.9790,0.3996,0.1533,0.5890,0.9567,0.1171,0.8639,0.9676,0.6630,0.0751,0.9306,0.1164,0.9642,0.2045,0.8220,0.9336,0.7713,0.9730,0.0888,0.3198,0.7545,0.6559,0.2449,0.9651,0.1130,0.5864,0.0869,0.8190,0.9134,0.4775,0.1042,0.9384,0.7759,0.9611,0.5851,0.9184,0.9597,0.1820,0.0966,0.1749,0.7356,0.9616,0.9544,0.5634,0.9286,0.4011,0.9617,0.9074,0.5325,0.5636,0.9747,0.1708,0.9521,0.4288,0.9574,0.1525,0.7340,0.0859,0.9644,0.9387,0.2598,0.4126,0.1906,0.9564,0.6151,0.9598,0.1171,0.9389,0.1482,0.0954,0.9612,0.2455,0.1570,0.4314,0.1059,0.8506,0.3947,0.8600,0.4350,0.5436,0.5620,0.8746,0.2830,0.7173,0.2178,0.5957,0.7070,0.5950,0.9131,0.9446,0.2062,0.9353,0.9770,0.9753,0.3864,0.9754,0.2610,0.7835,0.7963,0.9472,0.9768,0.9100,0.1418,0.1405,0.1375,0.4736,0.1193,0.2457,0.5015,0.1300,0.1957,0.9095,0.6263,0.9597,0.9431,0.2420,0.9697,0.1872,0.2640,0.1200,0.1794,0.9710,0.9122,0.1223,0.5518,0.8637,0.9725,0.8857,0.6104,0.4655,0.9179,0.8643,0.4011,0.7926,0.1793,0.9566,0.3033,0.7227,0.2500,0.9586,0.6989,0.9728,0.1952,0.1258,0.1796,0.2756,0.9742,0.9721,0.1850,0.9500,0.2375,0.1700,0.9592,0.8480,0.7854,0.1134,0.7314,0.4163,0.9712,0.9709,0.1649,0.9027,0.7331,0.2318,0.8928,0.4571,0.1317,0.9431,0.8064,0.3875,0.1830,0.2539,0.1168,0.1044,0.6598,0.2581,0.9567,0.9558,0.8354,0.9119,0.5616,0.1150,0.9231,0.1946,0.9059,0.1348,0.9239,0.1169,0.7972,0.8746,0.9706,0.1156,0.1189,0.8038,0.3426,0.9057,0.9751,0.3372,0.1128,0.3818,0.9364,0.9741,0.2592,0.9422,0.1509,0.0802,0.7543,0.4261,0.9361,0.1266,0.9261,0.7412,0.1942,0.4669,0.9748,0.2249,0.7525,0.2863,0.2499,0.2593,0.9698,0.2652,0.7663,0.1488,0.6108,0.7951,0.9580,0.9734,0.1888,0.9717,0.4336,0.5335,0.3614,0.2265,0.1921,0.1311,0.8355,0.1468,0.4474,0.9751,0.1950,0.1171,0.2979,0.9555,0.2460,0.1743,0.9660,0.0963,0.3964,0.9082,0.9697,0.6573,0.9329,0.8178,0.2685,0.2041,0.1343,0.2518,0.3610,0.1222,0.3856,0.1947,0.3321,0.2404,0.9748,0.1992,0.2340,0.1626,0.2522,0.4494,0.9196,0.7048,0.3793,0.1849,0.1557,0.2555,0.1577,0.3323,0.1290,0.4885,0.2152,0.1789,0.9733,0.2009,0.4155,0.1765,0.3316,0.3773,0.9439,0.1511,0.3426,0.2796,0.3739,0.2158,0.1656,0.4414,0.2718,0.3266,0.5206,0.1000,0.3661,0.2718,0.9121,0.7063,0.3343,0.8716,0.8470,0.3258,0.1532,0.2781,0.4234,0.2429,0.6851,0.9677,0.7454,0.3992,0.2186,0.2535,0.2580,0.6926,0.8232,0.2262,0.1309,0.2637,0.9620,0.1169,0.1470,0.1897,0.6434,0.5872,0.2741,0.3095,0.7978,0.3955,0.1849,0.2384,0.8827,0.1873,0.9692,0.9522,0.4581,0.1041,0.9062,0.2327,0.2944,0.0802,0.9717,0.5213,0.4355,0.2980,0.5700,0.6083,0.8736,0.9769,0.7057,0.3015,0.9577,0.1134,0.1822,0.8578,0.9602,0.8374,0.7130,0.7843,0.9641,0.9104,0.2381,0.9279,0.4356,0.8201,0.0771,0.9411,0.9647,0.2531,0.4412,0.8287,0.9723,0.4941,0.7206,0.1883,0.8184,0.9780,0.9614,0.4471,0.9739,0.5217,0.9448,0.7633,0.6998,0.2664,0.5607,0.1094,0.9665,0.1851,0.8493,0.1227,0.9303,0.5409,0.7978,0.9550,0.5697,0.9701,0.7962,0.6620,0.9628,0.9701,0.5745,0.3060,0.8879,0.0908,0.9531,0.9594,0.0698,0.9284,0.1732,0.1192,0.8748,0.1263,0.7139,0.9626,0.8111,0.8556,0.9682,0.1336,0.1589,0.1418,0.7000,0.1121,0.1330,0.1333,0.9263,0.1097,0.2361,0.9706,0.1977,0.9496,0.8893,0.9603,0.9573,0.3420,0.1932,0.7736,0.0742,0.6402,0.8750,0.9769,0.8423,0.2197,0.3738,0.7272,0.9610,0.8492,0.9099,0.9235,0.1078,0.8825,0.0921,0.1687,0.8912,0.1406,0.9422,0.1153,0.2681,0.9504,0.0968,0.9168,0.1451,0.9022,0.9481,0.9291,0.9069,0.1488,0.9749,0.0880,0.9095,0.3854,0.1499,0.7514,0.8379,0.9663,0.5419,0.8899,0.4865,0.0857,0.1716,0.4307,0.1083,0.8130,0.2287,0.1213,0.9565,0.8596,0.3704,0.6319,0.9769,0.2411,0.1658,0.9118,0.1909,0.9161,0.6709,0.9404,0.1302,0.8428,0.2274,0.4301,0.1483,0.1283,0.8800,0.1032,0.1976,0.2525,0.1106,0.8702,0.9440,0.9059,0.9504,0.8411,0.1535,0.8617,0.9518,0.2416,0.1797,0.1051,0.9338,0.9273,0.5062,0.9438,0.9427,0.9247,0.9601,0.9683,0.7857,0.0783,0.9035,0.8052,0.8467,0.9137,0.1317,0.1176,0.9693,0.9038,0.1457,0.9490,0.7361,0.9203,0.8804,0.9009,0.8939,0.8965,0.9606,0.9501,0.9559,0.7358,0.9227,0.3170,0.1554,0.2614,0.4911,0.8300,0.2440,0.9451,0.9571,0.1607,0.1713,0.8722,0.1459,0.9473,0.8801,0.9221,0.6225,0.9063,0.2829,0.1423,0.9272,0.9369,0.1651,0.9307,0.0888,0.1284,0.5070,0.2387,0.4273,0.9161,0.9140,0.1583,0.9149,0.9683,0.9208,0.6487,0.9588,0.1266,0.6676,0.8955,0.5655,0.9679,0.7944,0.6528,0.7549,0.9153,0.8128,0.6594,0.6284,0.6148,0.0873,0.0898,0.0744,0.0825,0.4651,0.9623,0.9647,0.1853,0.9476,0.6813,0.1483,0.9213,0.8366,0.1883,0.0954,0.1132,0.1101,0.7955,0.8979,0.7263,0.1398,0.9281,0.3257,0.3128,0.1400,0.1889,0.2249,0.1485,0.9169,0.2374,0.1996,0.8684,0.9214,0.0829,0.4100,0.0934,0.9250,0.2012,0.7580,0.2605,0.9185,0.4355,0.1205,0.2706,0.3503,0.2631,0.9390,0.7962,0.2231,0.5435,0.9323,0.8644,0.8888,0.3794,0.5185,0.4687,0.8111,0.7582,0.9769,0.9102,0.4604,0.7460,0.7779,0.5630,0.8562,0.2497,0.8961,0.7938,0.8124,0.8930,0.3275,0.1156,0.0740,0.1524,0.9174,0.2532,0.0937,0.8045,0.2570,0.8058,0.1052,0.1633,0.7804,0.1440,0.1716,0.7540,0.4099,0.6495,0.4099,0.3128,0.2674,0.1832,0.1369,0.5376,0.4402,0.5365,0.2512,0.4252,0.5382,0.5769,0.6624,0.3414,0.5690,0.4934,0.1071,0.7079,0.5689,0.7279,0.1844,0.8216,0.4343,0.8773,0.6840,0.5581,0.1333,0.3829,0.7047,0.8241,0.3814,0.4788,0.4564,0.3980,0.6014,0.1512,0.4822,0.7259,0.7935,0.1096,0.5358,0.1891,0.3912,0.2659,0.6238,0.2612,0.4792,0.4870,0.1957,0.2704,0.1627,0.0894,0.7811,0.6656,0.6445,0.5543,0.3742,0.5423,0.3454,0.1654,0.3339,0.1696,0.4996,0.1627,0.1608,0.8914,0.8383,0.8851,0.8716,0.7278,0.8020,0.8469,0.8253,0.8573,0.7009,0.7665,0.9033,0.9356,0.8679,0.7605,0.7550,0.6482,0.8748,0.4241,0.7710,0.7398,0.8775,0.7903,0.6483,0.6643,0.6068,0.4538,0.8779,0.8569,0.8775,0.2503,0.6636,0.8985,0.9232,0.8391,0.7141,0.6636,0.6428,0.4839,0.4566,0.2949,0.1954,0.7451,0.1842,0.1886,0.1783,0.8181,0.6416,0.2592,0.6120,0.2172,0.9169,0.8735,0.2408,0.2355,0.8155,0.2882,0.5977,0.1317,0.7695,0.7126,0.7141,0.2410,0.2877,0.8495,0.1736,0.7389,0.2579,0.2517,0.8416,0.3032,0.6600,0.1920,0.1551,0.6752,0.8587,0.1151,0.2223,0.7868,0.1810,0.7977,0.9331,0.6262,0.5932,0.8931,0.9265,0.8452,0.8451,0.1918,0.6450,0.7144,0.6008,0.1777,0.1878,0.2166,0.2298,0.6265,0.1593,0.2516,0.1032,0.1879,0.7486,0.4854,0.1988,0.2452,0.1448,0.9101,0.2916,0.2891,0.1041,0.2769,0.6217,0.1783,0.3657,0.9236,0.5076,0.5932,0.6461,0.2833,0.7643,0.4780,0.7457,0.4830,0.6792,0.4983,0.7024,0.4769,0.6444,0.3933,0.2317,0.4818,0.6356,0.2765,0.4142,0.4554,0.1937,0.1702,0.6461,0.8622,0.9187,0.8243,0.4626,0.5932,0.7263,0.6243,0.8151,0.6334,0.4990,0.5732,0.2905,0.2070,0.6548,0.7243,0.2611,0.3634,0.0918,0.1952,0.3247,0.5942,0.7976,0.3148,0.1552,0.1784,0.2166,0.3213,0.1620,0.3923,0.8335,0.7827,0.4532,0.7432,0.5769,0.1335,0.4901,0.3264,0.3216,0.7139,0.4762,0.2656,0.3812,0.5441,0.4814,0.3215,0.4626,0.6661,0.2025,0.1381,0.1873,0.1982,0.9094,0.4847,0.8921,0.8216,0.4691,0.6721,0.8975,0.8244,0.4775,0.3180,0.5727,0.5974,0.1907,0.4630,0.2689,0.3649,0.4829,0.3874,0.3263,0.3090,0.4645,0.1168,0.2045,0.4859,0.3157,0.5416,0.3277,0.4686,0.8256,0.3252,0.2912,0.6462,0.4859,0.7340,0.2726,0.1744,0.4420,0.1024,0.9180,0.3870,0.1703,0.1922,0.8805,0.6306,0.5483,0.3436,0.3238,0.1829,0.2189,0.3454,0.3561,0.1458,0.7772,0.3243,0.8087,0.2576,0.3690,0.2272,0.6747,0.5528,0.9031,0.2225,0.6860,0.1859,0.9055,0.3269,0.9602,0.6805,0.1942,0.5204,0.1338,0.3128,0.2924,0.6005,0.3448,0.3367,0.5897,0.2033,0.5677,0.5586,0.3331,0.4121,0.3594,0.4263,0.4406,0.6765,0.2547,0.3162,0.7557,0.5751,0.5762,0.7043,0.5958,0.8033,0.9186,0.7674,0.2954,0.2122,0.3105,0.8367,0.6022,0.3294,0.7735,0.1754,0.9653,0.6403,0.1290,0.1934,0.1857,0.4096,0.1851,0.2437,0.6550,0.6829,0.2829,0.5154,0.3876,0.5178,0.6125,0.5369,0.6575,0.9148,0.5626,0.4673,0.2889,0.3897,0.4409,0.1912,0.8219,0.1613,0.4362,0.7343,0.2928,0.3270,0.4494,0.5535,0.1222,0.5977,0.2549,0.5239,0.3213,0.6835,0.3884,0.6795,0.5294,0.0784,0.1198,0.6833,0.8376,0.4360,0.2045,0.2183,0.8779,0.2612,0.7540,0.4112,0.8843,0.1322,0.5155,0.4953,0.6925,0.8756,0.9426,0.5587,0.8441,0.4099,0.3260,0.4099,0.5131,0.5076,0.2759,0.2636,0.3064,0.4841,0.2255,0.2278,0.2788,0.6781,0.5073,0.3252,0.2390,0.2141,0.5752,0.2772,0.2213,0.4543,0.4779,0.3135,0.7279,0.7041,0.6345,0.2735,0.5907,0.5286,0.7303,0.5348,0.3179,0.3473,0.5696,0.7826,0.3197,0.3444,0.4250,0.2592,0.5727,0.3905,0.2807,0.4514,0.8633,0.2239,0.4228,0.4261,0.7890,0.6021,0.7346,0.5351,0.3745,0.3981,0.7701,0.7165,0.4513,0.7640,0.6954,0.8901,0.5013,0.4213,0.3147,0.6774,0.3183,0.2273,0.2311,0.3587,0.4106,0.5446,0.2487,0.2341,0.8555,0.3260,0.2142,0.3016,0.5372,0.5826,0.8002,0.6225,0.2927,0.4188,0.6719,0.5036,0.2341,0.6901,0.5983,0.2856,0.5785,0.8534,0.3194,0.2710,0.8918,0.4247,0.2977,0.3758,0.1520,0.6418,0.7382,0.3195,0.3928,0.7353,0.4404,0.3840,0.2882,0.5008,0.3709,0.2321,0.3229,0.2252,0.4201,0.7668,0.1871,0.3104,0.3450,0.4837,0.4787,0.5261,0.6422,0.4516,0.5004,0.4883,0.2026,0.4257,0.4935,0.2408,0.3889,0.6780,0.3284,0.5003,0.4261,0.4373,0.5771,0.2688,0.8438,0.5234,0.5006,0.4986,0.3979,0.5226,0.3946,0.3913,0.7896,0.3612,0.6936,0.3104,0.3393,0.5311,0.2891
    
]

In [ ]:
combination =""
for i in range(0,len(result_lgbm)):
    sayi=(result_lgbm[i]+result_mıl[i])/2
    combination=combination+","+"{:.4f}".format(sayi)
    
combination = combination[1:]
print(combination)

In [ ]:
temp2[0]